<h1>Monitorización de Base de Datos mediante Auditoría</h1>

<h2>Ejercicio N° 01: Utilizando tablas temporales de auditoría</h2>

Step 1 - connect this query window to your copy of AdventureWorksLT

Step 2 - create a system-versioned temporary table

In [1]:
CREATE TABLE dbo.Manager
( ManagerId int NOT NULL PRIMARY KEY,
  ManagerName nvarchar(50) NOT NULL,
  ManagerPassword varbinary(200) NOT NULL,
  ValidFrom datetime2 GENERATED ALWAYS AS ROW START NOT NULL, 
  ValidTo datetime2 GENERATED ALWAYS AS ROW END NOT NULL, 
  ChangedBy sysname NOT NULL CONSTRAINT DF_Employee_ChangedBy DEFAULT  (SUSER_SNAME()),
  PERIOD FOR SYSTEM_TIME (ValidFrom, ValidTo) 
) WITH (SYSTEM_VERSIONING = ON (HISTORY_TABLE = dbo.ManagerHistory));
GO

Commands completed successfully.

Total execution time: 00:00:00.211

Step 3 - insert example data

In [2]:
INSERT dbo.Manager (ManagerId, ManagerName, ManagerPassword)
VALUES (1, N'superuser', 0x3EED6B62548EC64A90E5D5D186FC9E5C),
(2, N'PeledYael', 0x19EF561D56A24D42A54169BD3DE23652),
(3, N'DavisSara', 0xD42025F8D7A67248AC36C5C8E955FA71);
GO

(3 rows affected)

Total execution time: 00:00:00.008

Step 4 - update a row

In [3]:
UPDATE dbo.Manager 
SET ManagerPassword = 0x3EED6B62548EC64A90E5D5D186FCFFFF,
ManagerName = 'administrator'
WHERE ManagerId = 1

(1 row affected)

Total execution time: 00:00:00.087

Step 5 - examine temporal table component tables

In [5]:
SELECT * FROM dbo.Manager;

SELECT * FROM dbo.ManagerHistory;

(3 rows affected)

(1 row affected)

Total execution time: 00:00:00.008

ManagerId,ManagerName,ManagerPassword,ValidFrom,ValidTo,ChangedBy
1,administrator,0x3EED6B62548EC64A90E5D5D186FCFFFF,2019-07-08 18:06:04.4132604,9999-12-31 23:59:59.9999999,MicrosoftAccount\jorgito_5_678@hotmail.com
2,PeledYael,0x19EF561D56A24D42A54169BD3DE23652,2019-07-08 18:06:00.7254230,9999-12-31 23:59:59.9999999,MicrosoftAccount\jorgito_5_678@hotmail.com
3,DavisSara,0xD42025F8D7A67248AC36C5C8E955FA71,2019-07-08 18:06:00.7254230,9999-12-31 23:59:59.9999999,MicrosoftAccount\jorgito_5_678@hotmail.com


ManagerId,ManagerName,ManagerPassword,ValidFrom,ValidTo,ChangedBy
1,superuser,0x3EED6B62548EC64A90E5D5D186FC9E5C,2019-07-08 18:06:00.7254230,2019-07-08 18:06:04.4132604,MicrosoftAccount\jorgito_5_678@hotmail.com


Step 6 - demonstrate FOR SYSTEM TIME ALL when querying a temporal table
ALL shows all data in both tables

In [6]:
SELECT * FROM dbo.Manager 
FOR SYSTEM_TIME ALL;

(4 rows affected)

Total execution time: 00:00:00.031

ManagerId,ManagerName,ManagerPassword,ValidFrom,ValidTo,ChangedBy
1,administrator,0x3EED6B62548EC64A90E5D5D186FCFFFF,2019-07-08 18:06:04.4132604,9999-12-31 23:59:59.9999999,MicrosoftAccount\jorgito_5_678@hotmail.com
2,PeledYael,0x19EF561D56A24D42A54169BD3DE23652,2019-07-08 18:06:00.7254230,9999-12-31 23:59:59.9999999,MicrosoftAccount\jorgito_5_678@hotmail.com
3,DavisSara,0xD42025F8D7A67248AC36C5C8E955FA71,2019-07-08 18:06:00.7254230,9999-12-31 23:59:59.9999999,MicrosoftAccount\jorgito_5_678@hotmail.com
1,superuser,0x3EED6B62548EC64A90E5D5D186FC9E5C,2019-07-08 18:06:00.7254230,2019-07-08 18:06:04.4132604,MicrosoftAccount\jorgito_5_678@hotmail.com


Step 7 - demonstrate FOR SYSTEM TIME AS OF when querying a temporal table
AS OF shows a point in time
Note that this returns the original data

In [7]:
DECLARE @t datetime2 = (SELECT TOP(1) ValidFrom FROM dbo.ManagerHistory WHERE ManagerId = 1)
SELECT * FROM dbo.Manager 
FOR SYSTEM_TIME AS OF @t
GO

(3 rows affected)

Total execution time: 00:00:00.040

ManagerId,ManagerName,ManagerPassword,ValidFrom,ValidTo,ChangedBy
2,PeledYael,0x19EF561D56A24D42A54169BD3DE23652,2019-07-08 18:06:00.7254230,9999-12-31 23:59:59.9999999,MicrosoftAccount\jorgito_5_678@hotmail.com
3,DavisSara,0xD42025F8D7A67248AC36C5C8E955FA71,2019-07-08 18:06:00.7254230,9999-12-31 23:59:59.9999999,MicrosoftAccount\jorgito_5_678@hotmail.com
1,superuser,0x3EED6B62548EC64A90E5D5D186FC9E5C,2019-07-08 18:06:00.7254230,2019-07-08 18:06:04.4132604,MicrosoftAccount\jorgito_5_678@hotmail.com


Step 8 - demonstrate that the history table cannot be edited (both commands will generate an error)

In [8]:
UPDATE dbo.ManagerHistory SET ChangedBy = 'maliciousUser';
GO
INSERT dbo.ManagerHistory (ManagerId, ManagerName, ManagerPassword)
VALUES (99, N'superuser', 0x3EED6B62548EC64A90E5D5D186FC9E5C)
GO

: Msg 13561, Level 16, State 1, Line 1
Cannot update rows in a temporal history table 'master.dbo.ManagerHistory'.

: Msg 13559, Level 16, State 1, Line 2
Cannot insert rows in a temporal history table 'master.dbo.ManagerHistory'.

Total execution time: 00:00:00.116

Step 9 - demonstrate that a user with sufficient permissions can insert misleading data into the ChangedBy column:

In [9]:
UPDATE dbo.Manager 
SET ManagerPassword = 0x0A0B,
ManagerName = 'hacked', ChangedBy = 'maliciousUser'
WHERE ManagerId = 1

(1 row affected)

Total execution time: 00:00:00.052

Step 10 - examine temporal table component tables

In [11]:
SELECT * FROM dbo.Manager;

SELECT * FROM dbo.ManagerHistory;

(3 rows affected)

(2 rows affected)

Total execution time: 00:00:00.020

ManagerId,ManagerName,ManagerPassword,ValidFrom,ValidTo,ChangedBy
1,hacked,0x0A0B,2019-07-08 18:11:02.5455849,9999-12-31 23:59:59.9999999,maliciousUser
2,PeledYael,0x19EF561D56A24D42A54169BD3DE23652,2019-07-08 18:06:00.7254230,9999-12-31 23:59:59.9999999,MicrosoftAccount\jorgito_5_678@hotmail.com
3,DavisSara,0xD42025F8D7A67248AC36C5C8E955FA71,2019-07-08 18:06:00.7254230,9999-12-31 23:59:59.9999999,MicrosoftAccount\jorgito_5_678@hotmail.com


ManagerId,ManagerName,ManagerPassword,ValidFrom,ValidTo,ChangedBy
1,superuser,0x3EED6B62548EC64A90E5D5D186FC9E5C,2019-07-08 18:06:00.7254230,2019-07-08 18:06:04.4132604,MicrosoftAccount\jorgito_5_678@hotmail.com
1,administrator,0x3EED6B62548EC64A90E5D5D186FCFFFF,2019-07-08 18:06:04.4132604,2019-07-08 18:11:02.5455849,MicrosoftAccount\jorgito_5_678@hotmail.com


Step 11 - tear down demonstration objects

In [12]:
ALTER TABLE dbo.Manager SET (SYSTEM_VERSIONING = OFF);
DROP TABLE dbo.Manager;
DROP TABLE dbo.ManagerHistory;

Commands completed successfully.

Total execution time: 00:00:00.069

<h2>Ejercicio N° 02: Utilizando auditorias</h2>

Step 1 - create an audit

In [20]:
USE master;
GO
CREATE SERVER AUDIT MIASQL_Audit 
    TO FILE (FILEPATH='C:\Data\')
    WITH (QUEUE_DELAY = 5000);
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.019

Step 2 - enable the audit

In [21]:
ALTER SERVER AUDIT MIASQL_Audit WITH (STATE = ON);
GO

Commands completed successfully.

Total execution time: 00:00:00.159

Step 3 - create a server audit specification

In [22]:
CREATE SERVER AUDIT SPECIFICATION AuditLogins
FOR SERVER AUDIT MIASQL_Audit
ADD (FAILED_LOGIN_GROUP),
ADD (SUCCESSFUL_LOGIN_GROUP)
WITH (STATE = ON);
GO

Commands completed successfully.

Total execution time: 00:00:00.061

Step 4 - create a database audit specification

In [23]:
USE salesapp1;
GO
CREATE DATABASE AUDIT SPECIFICATION salesapp1_audit_spec
FOR SERVER AUDIT MIASQL_Audit
ADD (INSERT,UPDATE ON DATABASE::salesapp1 BY public),
ADD (SELECT ON SCHEMA::HR BY public),
ADD (SCHEMA_OBJECT_CHANGE_GROUP)
WITH (STATE = ON);
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.115

Step 5 - alter the database audit specification

In [24]:
USE salesapp1
GO
ALTER DATABASE AUDIT SPECIFICATION salesapp1_audit_spec WITH (STATE = OFF);
GO
ALTER DATABASE AUDIT SPECIFICATION salesapp1_audit_spec
ADD (SCHEMA_OBJECT_PERMISSION_CHANGE_GROUP)
WITH (STATE = ON);
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.073

Step 6 - examine audit metadata

In [25]:
SELECT * FROM sys.server_audits;

(1 row affected)

Total execution time: 00:00:00.329

audit_id,name,audit_guid,create_date,modify_date,principal_id,type,type_desc,on_failure,on_failure_desc,is_state_enabled,queue_delay,predicate
65541,MIASQL_Audit,075849f2-4d9c-4d4c-acec-b6c5eaa6522c,2019-07-08 13:17:28.273,2019-07-08 13:17:28.273,1,FL,FILE,0,CONTINUE,1,5000,NULL


Step 7 - examine server audit specification metadata

In [26]:
SELECT * FROM sys.server_audit_specifications;

SELECT *
FROM sys.server_audit_specification_details AS sd
JOIN sys.dm_audit_actions AS aa
ON aa.name = sd.audit_action_name COLLATE Latin1_General_CI_AS_KS_WS

(1 row affected)

(2 rows affected)

Total execution time: 00:00:01.014

server_specification_id,name,create_date,modify_date,audit_guid,is_state_enabled
65536,AuditLogins,2019-07-08 13:17:44.980,2019-07-08 13:17:44.980,075849f2-4d9c-4d4c-acec-b6c5eaa6522c,1


server_specification_id,audit_action_id,audit_action_name,class,class_desc,major_id,minor_id,audited_principal_id,audited_result,is_group,action_id,name,class_desc,covering_action_name,parent_class_desc,covering_parent_action_name,configuration_level,containing_group_name,action_in_log
65536,LGFL,FAILED_LOGIN_GROUP,100,SERVER,0,0,2,SUCCESS AND FAILURE,1,LGFL,FAILED_LOGIN_GROUP,SERVER,NULL,NULL,NULL,Group,FAILED_LOGIN_GROUP,0
65536,LGSD,SUCCESSFUL_LOGIN_GROUP,100,SERVER,0,0,2,SUCCESS AND FAILURE,1,LGSD,SUCCESSFUL_LOGIN_GROUP,SERVER,NULL,NULL,NULL,Group,SUCCESSFUL_LOGIN_GROUP,0


Step 8 - examine database audit specification metadata

In [27]:
SELECT * FROM sys.database_audit_specifications;

SELECT *
FROM sys.database_audit_specification_details AS sd
JOIN sys.dm_audit_actions AS aa
ON aa.name = sd.audit_action_name COLLATE Latin1_General_CI_AS_KS_WS
AND aa.class_desc = sd.class_desc COLLATE Latin1_General_CI_AS_KS_WS

(1 row affected)

(5 rows affected)

Total execution time: 00:00:00.146

database_specification_id,name,create_date,modify_date,audit_guid,is_state_enabled
65536,salesapp1_audit_spec,2019-07-08 13:17:55.397,2019-07-08 13:17:55.397,075849f2-4d9c-4d4c-acec-b6c5eaa6522c,1


database_specification_id,audit_action_id,audit_action_name,class,class_desc,major_id,minor_id,audited_principal_id,audited_result,is_group,action_id,name,class_desc,covering_action_name,parent_class_desc,covering_parent_action_name,configuration_level,containing_group_name,action_in_log
65536,GRO,SCHEMA_OBJECT_PERMISSION_CHANGE_GROUP,0,DATABASE,0,0,0,SUCCESS AND FAILURE,1,GRO,SCHEMA_OBJECT_PERMISSION_CHANGE_GROUP,DATABASE,NULL,SERVER,SCHEMA_OBJECT_PERMISSION_CHANGE_GROUP,Group,SCHEMA_OBJECT_PERMISSION_CHANGE_GROUP,0
65536,IN,INSERT,0,DATABASE,0,0,0,SUCCESS AND FAILURE,0,IN,INSERT,DATABASE,SCHEMA_OBJECT_ACCESS_GROUP,SERVER,NULL,Action,SCHEMA_OBJECT_ACCESS_GROUP,0
65536,MNO,SCHEMA_OBJECT_CHANGE_GROUP,0,DATABASE,0,0,0,SUCCESS AND FAILURE,1,MNO,SCHEMA_OBJECT_CHANGE_GROUP,DATABASE,NULL,SERVER,SCHEMA_OBJECT_CHANGE_GROUP,Group,SCHEMA_OBJECT_CHANGE_GROUP,0
65536,UP,UPDATE,0,DATABASE,0,0,0,SUCCESS AND FAILURE,0,UP,UPDATE,DATABASE,SCHEMA_OBJECT_ACCESS_GROUP,SERVER,NULL,Action,SCHEMA_OBJECT_ACCESS_GROUP,0
65536,SL,SELECT,3,SCHEMA,5,0,0,SUCCESS AND FAILURE,0,SL,SELECT,SCHEMA,NULL,DATABASE,SELECT,Action,SCHEMA_OBJECT_ACCESS_GROUP,0


Step 9 - remove the audit 

In [28]:
USE master;
GO
ALTER SERVER AUDIT MIASQL_Audit WITH (STATE = OFF);
DROP SERVER AUDIT MIASQL_Audit;
GO

ALTER SERVER AUDIT SPECIFICATION AuditLogins WITH (STATE = OFF);
DROP SERVER AUDIT SPECIFICATION AuditLogins
GO

USE salesapp1;
GO
ALTER DATABASE AUDIT SPECIFICATION salesapp1_audit_spec WITH (STATE = OFF);
DROP DATABASE AUDIT SPECIFICATION salesapp1_audit_spec
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.080

<h2>Ejercicio N° 03: Utilizando auditorías personzalizadas</h2>

Step 1 - create an audit

In [29]:
USE master;
GO
CREATE SERVER AUDIT Custom_Audit 
    TO FILE (FILEPATH='C:\Data\')
    WITH (QUEUE_DELAY = 5000);
GO
ALTER SERVER AUDIT Custom_Audit WITH (STATE = ON);
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.009

Step 2 - create a server audit specification which includes the USER_DEFINED_AUDIT_GROUP action group

In [30]:
CREATE SERVER AUDIT SPECIFICATION UserDefinedEvents
FOR SERVER AUDIT Custom_Audit
ADD (USER_DEFINED_AUDIT_GROUP)
WITH (STATE = ON);
GO

Commands completed successfully.

Total execution time: 00:00:00.020

Step 3 - call sp_audit_write directly

In [31]:
EXEC sp_audit_write @user_defined_event_id = 999, 
                    @succeeded = 1,
                    @user_defined_information = N'Example call to sp_audit_write';

Commands completed successfully.

Total execution time: 00:00:00.045

Step 4 - demonstrate how a custom event appears in the audit 

In [33]:
SELECT user_defined_event_id, succeeded, user_defined_information
FROM sys.fn_get_audit_file ('C:\Data\Custom_Audit*',default,default)
WHERE user_defined_event_id = 999;


(1 row affected)

Total execution time: 00:00:00.036

user_defined_event_id,succeeded,user_defined_information
999,1,Example call to sp_audit_write


Step 5 - demonstrate the use of sp_audit_write in a stored procedure

In [34]:
USE salesapp1;
GO

CREATE PROC usp_OrderDetailDiscount
	@orderid int,
	@productid int,
	@discount numeric(4,3)
AS
	SET NOCOUNT ON

	IF @discount > 0.3
	BEGIN
		DECLARE @msg nvarchar(4000) = 
		  CONCAT('Order=',@orderid,':Product=',@productid,
		         ':discount=', @discount)

		
		EXEC sp_audit_write @user_defined_event_id = 998, 
				            @succeeded = 1,
						    @user_defined_information = @msg;
	END

	UPDATE Sales.OrderDetails
	SET discount = @discount
	WHERE orderid = @orderid
	AND productid = @productid
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.195

Step 6 - call the stored procedure twice the first call should not generate a custom audit event the second call should generate a custom audit event

In [35]:
EXEC dbo.usp_OrderDetailDiscount @orderid = 10248,@productid =	11, @discount = 0.05
EXEC dbo.usp_OrderDetailDiscount @orderid = 10248,@productid =	42, @discount = 0.45

Commands completed successfully.

Total execution time: 00:00:00.126

Step 7 - examine the audit data

In [36]:
SELECT user_defined_event_id, succeeded, user_defined_information
FROM sys.fn_get_audit_file ('C:\Data\Custom_Audit*',default,default)
WHERE user_defined_event_id = 998;

(1 row affected)

Total execution time: 00:00:00.037

user_defined_event_id,succeeded,user_defined_information
998,1,Order=10248:Product=42:discount=0.450


Step 8 - drop the audit

In [37]:
USE master;
GO
ALTER SERVER AUDIT Custom_Audit WITH (STATE = OFF);
DROP SERVER AUDIT Custom_Audit;
GO

ALTER SERVER AUDIT SPECIFICATION UserDefinedEvents WITH (STATE = OFF);
DROP SERVER AUDIT SPECIFICATION UserDefinedEvents
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.046

<h2>Ejercicio N° 04: Administrando auditorías</h2>

Step 1 - create an audit with a file target

In [38]:
USE master;
GO
CREATE SERVER AUDIT File_Audit 
    TO FILE (FILEPATH='C:\Data\')
    WITH (QUEUE_DELAY = 5000);
GO
ALTER SERVER AUDIT File_Audit WITH (STATE = ON);
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.009

Step 2 - create an audit with a Windows application log target

In [39]:
CREATE SERVER AUDIT AppLog_Audit 
    TO APPLICATION_LOG
    WITH (QUEUE_DELAY = 5000);
GO
ALTER SERVER AUDIT AppLog_Audit WITH (STATE = ON);
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.004

Step 3 - add the same database audit specification to both audits

In [40]:
USE salesapp1;
GO
CREATE DATABASE AUDIT SPECIFICATION sales_select_spec_file
FOR SERVER AUDIT File_Audit
ADD (SELECT ON SCHEMA::Sales BY public)
WITH (STATE = ON);
GO

CREATE DATABASE AUDIT SPECIFICATION sales_select_spec_applog
FOR SERVER AUDIT AppLog_Audit
ADD (SELECT ON SCHEMA::Sales BY public)
WITH (STATE = ON);
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.022

Step 4 - execute a simple select statement which matches the audit specification

In [41]:
SELECT TOP 10 * FROM Sales.Customers;
GO

(10 rows affected)

Total execution time: 00:00:00.093

custid,companyname,contactname,contacttitle,address,city,region,postalcode,country,phone,fax
1,Customer NRZBB,"Allen, Michael",Sales Representative,Obere Str. 0123,Berlin,NULL,10092,Germany,030-3456789,030-0123456
2,Customer MLTDN,"Hassall, Mark",Owner,Avda. de la Constitución 5678,México D.F.,NULL,10077,Mexico,(5) 789-0123,(5) 456-7890
3,Customer KBUDE,"Peoples, John",Owner,Mataderos 7890,México D.F.,NULL,10097,Mexico,(5) 123-4567,NULL
4,Customer HFBZG,"Arndt, Torsten",Sales Representative,7890 Hanover Sq.,London,NULL,10046,UK,(171) 456-7890,(171) 456-7891
5,Customer HGVLZ,"Higginbotham, Tom",Order Administrator,Berguvsvägen 5678,Luleå,NULL,10112,Sweden,0921-67 89 01,0921-23 45 67
6,Customer XHXJV,"Poland, Carole",Sales Representative,Forsterstr. 7890,Mannheim,NULL,10117,Germany,0621-67890,0621-12345
7,Customer QXVLA,"Bansal, Dushyant",Marketing Manager,"2345, place Kléber",Strasbourg,NULL,10089,France,67.89.01.23,67.89.01.24
8,Customer QUHWH,"Ilyina, Julia",Owner,"C/ Araquil, 0123",Madrid,NULL,10104,Spain,(91) 345 67 89,(91) 012 34 56
9,Customer RTXGC,"Raghav, Amritansh",Owner,"6789, rue des Bouchers",Marseille,NULL,10105,France,23.45.67.89,23.45.67.80
10,Customer EEALV,"Bassols, Pilar Colome",Accounting Manager,8901 Tsawassen Blvd.,Tsawassen,BC,10111,Canada,(604) 901-2345,(604) 678-9012


Step 5 - examine the file-based audit output. Demonstrate some of the most useful fields

In [43]:
SELECT *
FROM sys.fn_get_audit_file ('C:\Data\File_Audit*',default,default)

(2 rows affected)

Total execution time: 00:00:00.048

event_time,sequence_number,action_id,succeeded,permission_bitmask,is_column_permission,session_id,server_principal_id,database_principal_id,target_server_principal_id,target_database_principal_id,object_id,class_type,session_server_principal_name,server_principal_name,server_principal_sid,database_principal_name,target_server_principal_name,target_server_principal_sid,target_database_principal_name,server_instance_name,database_name,schema_name,object_name,statement,additional_information,file_name,audit_file_offset,user_defined_event_id,user_defined_information,audit_schema_version,sequence_group_id,transaction_id,client_ip,application_name,duration_milliseconds,response_rows,affected_rows
2019-07-08 18:23:21.9568698,1,AUSC,1,0x00000000000000000000000000000000,0,60,259,0,0,0,0,A,MicrosoftAccount\jorgito_5_678@hotmail.com,MicrosoftAccount\jorgito_5_678@hotmail.com,0x010500000000000515000000771C9BC3AE1B42589774B81FE9030000,,,NULL,,DESKTOP-SKNOLPR,,,,,"<action_info xmlns=""http://schemas.microsoft.com/sqlserver/2008/sqlaudit_data""><session><![CDATA[File_Audit$A]]></session><action>event enabled</action><startup_type>manual</startup_type><object><![CDATA[audit_event]]></object></action_info>",C:\Data\File_Audit_EC635055-1FFC-4A2B-B670-A1AF061A2CAF_0_132070838019540000.sqlaudit,5632,0,,1,0x00000000000000000000000000000000,0,local machine,,0,0,0
2019-07-08 18:24:10.5759528,1,SL,1,0x00000000000000000000000000000001,1,60,259,1,0,0,485576768,U,MicrosoftAccount\jorgito_5_678@hotmail.com,MicrosoftAccount\jorgito_5_678@hotmail.com,0x010500000000000515000000771C9BC3AE1B42589774B81FE9030000,dbo,,NULL,,DESKTOP-SKNOLPR,salesapp1,Sales,Customers,SELECT TOP 10 * FROM Sales.Customers,,C:\Data\File_Audit_EC635055-1FFC-4A2B-B670-A1AF061A2CAF_0_132070838019540000.sqlaudit,7168,0,,1,0x08BD7E3CD79B9043A107B923D4E6A437,466762,local machine,azdata-Query,0,0,0


Step 6 - examin the Windows application log audit output. Use Event Viewer

Step 7 - drop audits and audit specifications, notice that the audits and specifications must be disabled before they can be dropped

In [44]:
ALTER DATABASE AUDIT SPECIFICATION sales_select_spec_applog WITH (STATE = OFF);
DROP DATABASE AUDIT SPECIFICATION sales_select_spec_applog;
ALTER DATABASE AUDIT SPECIFICATION sales_select_spec_file WITH (STATE = OFF);
DROP DATABASE AUDIT SPECIFICATION sales_select_spec_file;
GO
USE master;
ALTER SERVER AUDIT AppLog_Audit WITH (STATE = OFF);
DROP SERVER AUDIT AppLog_Audit;
ALTER SERVER AUDIT File_Audit WITH (STATE = OFF);
DROP SERVER AUDIT File_Audit;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.033